## Lesson 23

GOAL: Create UNet finaly

In this lesson we will use it for super resolution

### Tiny Image net

64x64 images [tiny-imagenet](https://paperswithcode.com/dataset/tiny-imagenet)

* Notebook 24_imgnet_tiny.ipynb was discussed first.  (I note there are 3 other closly named notebooks though in the [repo](https://github.com/fastai/course22p2/tree/master))

* As in last week, my notes will be sparse as I don't intend to reproduce code here. I will just note the key points.

* TinyData set has labels like `n01443537` which are not human readable, they are some kind of encoded category. ('wordnet'). There are 200 categories. In the notebook they map these to integers 0-199.   There is also a words.txt file which turns these into human readable names.  Wordnet is some kind of hierarchical category system. 

* First part of the lesson focused just on setting up the data loaders for training and validation.

* Also data augmetnation was added, setting up random crop / horizinoal flip / and random erase.

#### Basic model 

* use the fashionmnist model with dropout (Resblocks) (Details in the notebook)
* Trains it for 25 epochs on classification task to get 60% accuracy (Training is at 91...so some overfitting)

#### Papers with code - how good can we do?

* [Tiny IMageNet Classification](https://paperswithcode.com/sota/image-classification-on-tiny-imagenet-1) shows results up to 93% on validation.  (Some of the models are using pretrained models, but at the current time it looks like the top performaer is not) 

* Highest resnet type model seems to get abotu 72% accruracy.  Oddly the paper link is missing now (was linked to the paper Jeremy looked at).  I found the paper was renamed to [Harnessing Hard Mixed Samples with Decoupled Regularizer](https://arxiv.org/abs/2203.10761)  Paper has to do with a proposal of new data augmentation technique. 

* Jeremy then makes a deeper model with multiple resblocks before downsampling. (i.e. multiple resblocks where there was one previously). This gets accuracy up to 62%.

*  Jeremy then adds *more* augmentation using something called [Trivial Augment](https://arxiv.org/abs/2103.10158) . This is parameter free and gives good results. It randomly picks a different augmentation for each image.  This is built into pytorch. `TrivialAugmentWide` is the version that is used.  Looking at teh source it picks from a wide range of augmentations, including Shear, Translate, Rotate, Brightness, etc. with random strength.  Unlike the others, this doesn't work well if the same transform is provided to all images in the batch.  So jeremy shows how to apply at a per item level. 

* Next he reorders the identiy as in   [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)  This is a small change  where the residual is added after the activation. This is called "Pre-activation" resblock beacuse the activation is applied before the residual is added. (This is supposed work better for *deeper* models).

* This results in a model that gets 65% accuracy after 50 epochs.  67.5% with 200 epochs.  This is basically the state of the art for these kinds of models


### Super Resolution
49 Minutes into the video

* Independant variable will be a 32x32 scaled down image, and the dependent variable will be the (origina) 64x64 image.

Notebook 25_superres.ipynb

